Laplace on surfaces
====

In [ ]:
from ngsolve import *
from netgen.csg import *
import netgen.meshing as meshing
import netgen.gui
%gui tk

Define geometry and $H^1$ space

In [ ]:
geo = CSGeometry()
sphere = Sphere (Pnt(0,0,0),1).bc("sphere")
geo.Add(sphere)
mesh = Mesh(geo.GenerateMesh(maxh=0.2, perfstepsend=meshing.MeshingStep.MESHSURFACE))

order = 3
mesh.Curve(order)
Draw(mesh)

fes = H1(mesh, order=order)
print ("ndof-H1:", fes.ndof)

\begin{align*}
\int_S\nabla_{\Gamma}u\cdot\nabla_{\Gamma}v + uv\,ds = f(v)
\end{align*}

In [ ]:
u,v = fes.TnT()

a = BilinearForm(fes, symmetric=True)
a += SymbolicBFI ( grad(u)*grad(v) + u*v, BND )
a.Assemble()

f = LinearForm(fes)
f += SymbolicLFI ( x*sin(y)*v, BND )
f.Assemble()

gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec

Draw (gfu, mesh, "gfu")